In [1]:
#Cargando librerías de manipulación y visualización de datos
import pandas as pd
from pathlib import Path
import sys
import numpy as np
# Usamos la carpeta actual del notebook como punto de partida
   # sube un nivel desde notebooks/
root_dir = Path().resolve().parent
sys.path.append(str(root_dir))

from codes.feature_engineering import FeatureEngineering

feature_engineer = FeatureEngineering()

In [2]:
df = feature_engineer.lectura_csv()

2025-10-02 17:40:57,674 [INFO] Leyendo CSV desde: /home/cesar/Escritorio/Challenge Meli/test_meli/data/data_proccesed.csv


In [3]:
df_ = df.copy()
df_['seller_id'] = df_['seller_id'].astype(str)

In [4]:
df_1 = feature_engineer.eliminar_id_title(df_)

In [5]:
df_2 = feature_engineer.indicativo_descuento(df_1)

Ningún producto presenta descuento


In [6]:
#Limpiando tildes, espacios, mayusculas en variables categoricas para buscar realizar paretos
df_3 = feature_engineer.limpiar_dataframe(df_2,['seller_country','seller_province','seller_city'])

In [7]:
columns_categorica = df_3.select_dtypes(include=["object", "string"]).columns.tolist()
df_4 = feature_engineer.creando_paretos(df_3,0.8,columns_categorica)

In [8]:
lista_drop = feature_engineer.analizando_labels(df_4)

La varible seller_loyalty será incluída en el análisis
La varible buying_mode será incluída en el análisis
La varible shipping_mode será incluída en el análisis
La varible status será incluída en el análisis


In [9]:
df_5 = df_4.drop(columns=lista_drop)

In [10]:
'''Al tener redundancia en las variables bae_price y price eliminaremos una de las dos, así mismo la variable available_quantity que es el resultado
de la diferencia entre inicial y sold.
'''
df_6 = df_5.drop(columns=['base_price','available_quantity'])
df_6['is_new'] = df_6['is_new'].astype(str)
df_6['outlier_price'] = df_6['outlier_price'].astype(str)

In [11]:
'''Ahora realizaré un análisis de que variables podrían ser features para llevar a cabo un modelo predictivo.
Decidí tomar la variable sold_quantity como variable objetivo, ya que, considero que predecir la cantidad de producto que será vendida
es útil para tener prevención y suficiente inventario de la misma.
'''
df_6.head(10)

,price,seller_loyalty,buying_mode,shipping_mode,shipping_admits_pickup,shipping_is_free,status,is_new,initial_quantity,sold_quantity,outlier_price
0,6700.00,silver,buy_it_now,custom,True,False,active,0,1.0,0.0,1
1,119.00,bronze,buy_it_now,not_specified,False,False,active,1,19.0,0.0,0
2,450.00,bronze,buy_it_now,me2,False,False,active,1,2.0,0.0,0
3,40.00,bronze,buy_it_now,me2,True,False,active,0,1.0,0.0,0
4,30.00,bronze,buy_it_now,not_specified,True,False,active,1,1.0,1.0,0
5,1700.00,silver,buy_it_now,me2,True,False,active,1,1.0,0.0,0
6,650.00,bronze,buy_it_now,me2,True,False,active,0,1.0,0.0,0
7,350.00,bronze,buy_it_now,not_specified,True,False,active,1,1.0,0.0,0
8,620.00,bronze,buy_it_now,not_specified,True,False,active,1,7.0,3.0,0
9,45.73,bronze,buy_it_now,me2,False,False,active,0,1.0,0.0,0


In [12]:
#Se realiza un test de diferencias de medias para las variables categoricas, las que no sean significativas estadísticamente serán borradas
salida, drop_columns_categoricas = feature_engineer.test_diferencia_medias(df_6)

In [13]:
#Correlación de Pearson y Spearman para evaluar la relación entre las variables numéricas con la variable objetivo
salida_num, drop_columns_numericas = feature_engineer.calcular_correlaciones(df_6)

In [14]:
df_7 = df_6.drop(columns=drop_columns_categoricas+drop_columns_numericas)
df_7.head(10)

,seller_loyalty,buying_mode,shipping_mode,shipping_admits_pickup,shipping_is_free,is_new,sold_quantity,outlier_price
0,silver,buy_it_now,custom,True,False,0,0.0,1
1,bronze,buy_it_now,not_specified,False,False,1,0.0,0
2,bronze,buy_it_now,me2,False,False,1,0.0,0
3,bronze,buy_it_now,me2,True,False,0,0.0,0
4,bronze,buy_it_now,not_specified,True,False,1,1.0,0
5,silver,buy_it_now,me2,True,False,1,0.0,0
6,bronze,buy_it_now,me2,True,False,0,0.0,0
7,bronze,buy_it_now,not_specified,True,False,1,0.0,0
8,bronze,buy_it_now,not_specified,True,False,1,3.0,0
9,bronze,buy_it_now,me2,False,False,0,0.0,0


In [15]:
df_7['is_new'] = df_7['is_new'].astype(int)
df_7['outlier_price'] = df_7['outlier_price'].astype(int)

In [16]:
#Booleanas a numericas
df_8 = feature_engineer.bool_to_number(df_7)

In [17]:
#Por último convertiremos en dummy las variables categoricas ya que es necesario tenerlas en formato numérico para ser utilizados dentro de un modelo 
df_final = feature_engineer.dummies(df_8)

In [18]:
feature_engineer.guardar_csv(df_final)

2025-10-02 17:41:22,054 [INFO] Guardando CSV a la ruta: /home/cesar/Escritorio/Challenge Meli/test_meli/data/data_to_model.csv
